# On the pertinence of LLMs for ontology learning: results analysis

This notebook contains the functions we used to construct our results. 
We try to automate as much as possible our analysis, particularly leveraging some SPARQL queries.
However, human are still needed to align the ontologies constructed with our base one, i.e., the pizza ontology.

In particular, we needed to align the ontologies independently of the exact labels.
Hence, to help us in this endeavour, we used a basic textual distance on the labels or end of the URI if no label were available.
In the end, the counting was done manually. 

## Notebook setup

In [1]:
import os

from rapidfuzz import process, fuzz
from rdflib import Graph, BNode
from rdflib.namespace import Namespace

import pandas as pd

In [2]:
# Define some namespaces to ease the URIs manipulation and visualisation

olaf_eswc_ns = Namespace("https://github.com/wikit-ai/olaf-llm-eswc2024/o/example#")
pizza_onto_ns = Namespace("http://www.co-ode.org/ontologies/pizza/pizza.owl#")

olaf_eswc_ns_bindings = {
    "olaf-eswc": olaf_eswc_ns,
    "pizza-onto": pizza_onto_ns
}

In [3]:
# Load the different RDF graphs

pizza_onto_graph = Graph()
pizza_onto_graph.parse(os.path.join(os.getenv("DATA_PATH"), "pizza_onto_ground_truth.ttl"))

llm_text2owl_graph = Graph()
llm_text2owl_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "llm_text_to_owl", "llm_owl_pizza_onto_eswc2024.ttl"))

olaf_llm_graph = Graph()
olaf_llm_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "llm_pipeline", "llm_pipeline_kr_rdf_graph_eswc2024.ttl"))

olaf_no_llm_graph = Graph()
olaf_no_llm_graph.parse(os.path.join(os.getenv("RESULTS_PATH"), "no_llm_pipeline", "no_llm_pipeline_kr_rdf_graph_eswc2024.ttl"))

<Graph identifier=N57fab2182d72433eb47bcd7f13f8a0ea (<class 'rdflib.graph.Graph'>)>

## SPARQL tools

In [4]:
# Some SPARQL queries we use for our analysis

owl_classes_sparql_q = """
            SELECT DISTINCT ?class WHERE {
                ?class rdf:type owl:Class .
            }
        """

owl_classes_labels_sparql_q = """
            SELECT ?label WHERE {
                ?class rdf:type owl:Class ;
                        rdfs:label ?label .
            }
        """

owl_classes_en_labels_sparql_q = """
            SELECT ?label WHERE {
                ?class rdf:type owl:Class ;
                        rdfs:label ?label .
                FILTER(LANG(?label) = "en").
            }
        """

owl_obj_props_sparql_q = """
            SELECT DISTINCT ?prop WHERE {
                ?prop rdf:type owl:ObjectProperty .
            }
        """

owl_named_individuals_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                ?ind rdf:type owl:NamedIndividual .
            }
        """

owl_named_ind_from_obj_props_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?ind ?prop ?o .
                }
                UNION
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?s ?prop ?ind .
                }
                UNION
                {
                    ?ind rdf:type ?class .
                    ?class rdf:type owl:Class .
                }
                UNION
                {
                    ?ind rdf:type owl:NamedIndividual .
                }
            }
        """

owl_named_classes_ind_from_obj_props_sparql_q = """
            SELECT DISTINCT ?ind WHERE {
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?ind ?prop ?o .
                }
                UNION
                {
                    ?prop rdf:type owl:ObjectProperty .
                    ?s ?prop ?ind .
                }
                UNION
                {
                    ?ind rdf:type ?class .
                    ?class rdf:type owl:Class .
                }
                UNION
                {
                    ?ind rdf:type owl:NamedIndividual .
                }
                UNION
                {
                    ?ind rdf:type owl:Class .
                }
            }
        """

rdfs_subclassof_tuples_sparql_q = """
            SELECT DISTINCT ?child ?parent WHERE {
                ?child rdfs:subClassOf ?parent .
            }
        """

In [5]:
def get_sparql_q_res_fragments_skip_bnodes(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs fragments skipping blank nodes.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)
    
    fragments = set()
    for res in q_res:
        t = []
        for item in res:
            if not isinstance(item, BNode):
                t.append(item.fragment)
        fragments.add(tuple(t))

    return fragments

In [6]:
def get_sparql_q_label_res(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs labels.

    The SPARQL query must return labels, i.e., strings.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)

    labels = {str(res[0]) for res in q_res}

    return labels

In [7]:
def get_sparql_q_tuple_res(sparql_q: str, graph: Graph, ns: dict[str, Namespace]) -> set[tuple]:
    """Run a SPARQL query over an RDF graph and return the URIs labels.

    The SPARQL query must return labels, i.e., strings.

    Parameters
    ----------
    sparql_q : str
        The SPARQL query.
    graph : Graph
        The RDF graph.
    ns : dict[str, Namespace]
        Namespaces to use to extract URIs fragments (i.e., end).

    Returns
    -------
    set[tuple]
        The results.
    """
    q_res = graph.query(sparql_q, initNs=ns)

    labels = {(str(res[0]), str(res[1])) for res in q_res}

    return labels

## Ontologies overview

In [8]:
# Extract ontology components

llm_text2owl_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_classes = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_classes_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_individuals = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_individuals_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_individuals_from_props = get_sparql_q_res_fragments_skip_bnodes(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

llm_text2owl_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_g_subclassof_t = get_sparql_q_res_fragments_skip_bnodes(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)

In [9]:
# Construct the counts table for visualisation

onto_overview = {
    "Count": [
        "OWL named classes", 
        "OWL object properties", 
        "OWL named individuals", 
        "OWL named individuals from object properties", 
        "RDFS subClassOf tuples"
    ],
    "Pizza Ontology": [
        len(pizza_onto_g_classes), 
        len(pizza_onto_g_props), 
        len(pizza_onto_g_individuals), 
        len(pizza_onto_g_individuals_from_props), 
        len(pizza_onto_g_subclassof_t)
    ],
    "Text to OWL": [
        len(llm_text2owl_g_classes), 
        len(llm_text2owl_g_props), 
        len(llm_text2owl_g_individuals), 
        len(llm_text2owl_g_individuals_from_props), 
        len(llm_text2owl_g_subclassof_t)
    ],
    "OLAF LLM": [
        len(olaf_llm_g_classes), 
        len(olaf_llm_g_props), 
        len(olaf_llm_g_individuals), 
        len(olaf_llm_g_individuals_from_props), 
        len(olaf_llm_g_subclassof_t)
    ],
    "OLAF no LLM": [
        len(olaf_no_llm_g_classes), 
        len(olaf_no_llm_g_props), 
        len(olaf_no_llm_g_individuals), 
        len(olaf_no_llm_g_individuals_from_props), 
        len(olaf_no_llm_g_subclassof_t)
    ]
}

df_overview = pd.DataFrame(onto_overview)

In [10]:
df_overview

,Count,Pizza Ontology,Text to OWL,OLAF LLM,OLAF no LLM
0,OWL named classes,97,36,99,111
1,OWL object properties,8,2,77,22
2,OWL named individuals,5,0,0,343
3,OWL named individuals from object properties,5,27,97,343
4,RDFS subClassOf tuples,141,33,114,390


## Ontologies evaluation

Classes

In [11]:
pizza_onto_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_en_labels_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_class_labels = {label.lower().replace(" ", "") for label in pizza_onto_class_labels}

In [12]:
llm_text2owl_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_labels_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_class_labels = list({label.lower().replace(" ", "") for label in llm_text2owl_class_labels})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, llm_text2owl_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [13]:
common_classes_pizza_text2owl=36

In [14]:
olaf_llm_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_class_labels = list({label.lower().replace("https://github.com/wikit-ai/olaf-llm-eswc2024/o/example#", "") for label in olaf_llm_class_labels})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, olaf_llm_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [15]:
common_classes_pizza_olaf_llm=55

In [16]:
olaf_no_llm_class_labels = get_sparql_q_label_res(sparql_q=owl_classes_labels_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_class_labels = list({label.lower().replace(" ", "") for label in olaf_no_llm_class_labels})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_class_labels:
#     res = process.extract(true_label, olaf_no_llm_class_labels, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [17]:
common_classes_pizza_olaf_no_llm=43

Individuals

In [18]:
pizza_onto_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_ind_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_ind_uri})

In [19]:
llm_text2owl_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_ind_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_ind_uri:
#     res = process.extract(true_label, llm_text2owl_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [20]:
common_ind_pizza_text2owl=0

In [21]:
olaf_llm_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_ind_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_ind_uri:
#     res = process.extract(true_label, olaf_llm_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

In [22]:
common_ind_pizza_olaf_llm = 1

In [23]:
olaf_no_llm_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_ind_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_ind_uri:
#     res = process.extract(true_label, olaf_no_llm_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})")

In [24]:
common_ind_pizza_olaf_no_llm = 2

Classes + indiviuals

In [25]:
pizza_onto_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_classes_ind_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_classes_ind_uri})
pizza_onto_classes_ind_uri = [uri for uri in pizza_onto_classes_ind_uri if not(uri[0:2]=="nc")]

In [26]:
llm_text2owl_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, llm_text2owl_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [27]:
common_classes_ind_pizza_text2owl=34

In [28]:
olaf_llm_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, olaf_llm_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [29]:
common_classes_ind_pizza_olaf_llm=54

In [30]:
olaf_no_llm_classes_ind_uri = get_sparql_q_label_res(sparql_q=owl_named_classes_ind_from_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_classes_ind_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_classes_ind_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_classes_ind_uri:
#     res = process.extract(true_label, olaf_no_llm_classes_ind_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

In [31]:
common_classes_ind_pizza_olaf_no_llm=59

Object properties

In [32]:
pizza_onto_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_obj_prop_uri = list({label.lower().replace(str(pizza_onto_ns), "") for label in pizza_onto_obj_prop_uri})

In [33]:
llm_text2owl_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in llm_text2owl_obj_prop_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_obj_prop_uri:
#     res = process.extract(true_label, llm_text2owl_obj_prop_uri, scorer=fuzz.WRatio, limit=2)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]})") 
    

In [34]:
common_obj_prop_pizza_text2owl=2

In [35]:
olaf_llm_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_llm_obj_prop_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_obj_prop_uri:
#     res = process.extract(true_label, olaf_llm_obj_prop_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

In [36]:
common_obj_prop_pizza_olaf_llm = 5

In [37]:
olaf_no_llm_obj_prop_uri = get_sparql_q_label_res(sparql_q=owl_obj_props_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_obj_prop_uri = list({label.lower().replace(str(olaf_eswc_ns), "") for label in olaf_no_llm_obj_prop_uri})

# Uncomment this to visualise the string alignments
# for true_label in pizza_onto_obj_prop_uri:
#     res = process.extract(true_label, olaf_no_llm_obj_prop_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 

In [38]:
common_obj_prop_pizza_olaf_no_llm = 3

SubClassOf tuples

In [39]:
pizza_onto_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=pizza_onto_graph, ns=olaf_eswc_ns_bindings)
pizza_onto_sub_uri = list({f"{label[0].lower().replace(str(pizza_onto_ns), '')}#{label[1].lower().replace(str(pizza_onto_ns), '')}" for label in pizza_onto_sub_uri if not(label[0][0:2]=="nc" or label[1][0:2]=="nc")})

In [40]:
llm_text2owl_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=llm_text2owl_graph, ns=olaf_eswc_ns_bindings)
llm_text2owl_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in llm_text2owl_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, llm_text2owl_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [41]:
common_sub_pizza_text2owl=17

In [42]:
olaf_llm_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_llm_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in olaf_llm_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, olaf_llm_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [43]:
common_sub_pizza_olaf_llm=27

In [44]:
olaf_no_llm_sub_uri = get_sparql_q_tuple_res(sparql_q=rdfs_subclassof_tuples_sparql_q, graph=olaf_no_llm_graph, ns=olaf_eswc_ns_bindings)
olaf_no_llm_sub_uri = list({f"{label[0].lower().replace(str(olaf_eswc_ns), '')}#{label[1].lower().replace(str(olaf_eswc_ns), '')}" for label in olaf_no_llm_sub_uri})

# Uncomment this for loop to visualise the string alignments
# for true_label in pizza_onto_sub_uri:
#     res = process.extract(true_label, olaf_no_llm_sub_uri, scorer=fuzz.WRatio, limit=3)
#     print(f"{true_label} :  {res[0][0]} ({res[0][1]}), {res[1][0]} ({res[1][1]}), {res[2][0]} ({res[2][1]})") 
    

In [45]:
common_sub_pizza_olaf_no_llm=6

Results

In [46]:
classes_precision_text2owl = common_classes_pizza_text2owl/len(llm_text2owl_class_labels)
classes_precision_olafLLM =  common_classes_pizza_olaf_llm/len(olaf_llm_class_labels)
classes_precision_olafNoLLM = common_classes_pizza_olaf_no_llm/len(olaf_no_llm_class_labels)
  
classes_recall_text2owl = common_classes_pizza_text2owl/len(pizza_onto_class_labels)
classes_recall_olafLLM = common_classes_pizza_olaf_llm/len(pizza_onto_class_labels)
classes_recall_olafNoLLM = common_classes_pizza_olaf_no_llm/len(pizza_onto_class_labels)

individuals_precision_text2owl = common_ind_pizza_text2owl/len(llm_text2owl_ind_uri)
individuals_precision_olafLLM =  common_ind_pizza_olaf_llm/len(olaf_llm_ind_uri)
individuals_precision_olafNoLLM = common_ind_pizza_olaf_no_llm/len(olaf_no_llm_ind_uri)

individuals_recall_text2owl = common_ind_pizza_text2owl/len(pizza_onto_ind_uri)
individuals_recall_olafLLM = common_ind_pizza_olaf_llm/len(pizza_onto_ind_uri)
individuals_recall_olafNoLLM = common_ind_pizza_olaf_no_llm/len(pizza_onto_ind_uri)

classesAndIndividuals_precision_text2owl = common_classes_ind_pizza_text2owl/len(llm_text2owl_classes_ind_uri)
classesAndIndividuals_precision_olafLLM = common_classes_ind_pizza_olaf_llm/len(olaf_llm_classes_ind_uri)
classesAndIndividuals_precision_olafNoLLM = common_classes_ind_pizza_olaf_no_llm/len(olaf_no_llm_classes_ind_uri)

classesAndIndividuals_recall_text2owl = common_classes_ind_pizza_text2owl/len(pizza_onto_classes_ind_uri)
classesAndIndividuals_recall_olafLLM = common_classes_ind_pizza_olaf_llm/len(pizza_onto_classes_ind_uri)
classesAndIndividuals_recall_olafNoLLM = common_classes_ind_pizza_olaf_no_llm/len(pizza_onto_classes_ind_uri)

objectProperties_precision_text2owl = common_obj_prop_pizza_text2owl/len(pizza_onto_obj_prop_uri)
objectProperties_precision_olafLLM = common_obj_prop_pizza_olaf_llm/len(olaf_llm_obj_prop_uri)
objectProperties_precision_olafNoLLM = common_obj_prop_pizza_olaf_no_llm/len(olaf_no_llm_obj_prop_uri)

objectProperties_recall_text2owl = common_obj_prop_pizza_text2owl/len(llm_text2owl_obj_prop_uri)
objectProperties_recall_olafLLM = common_obj_prop_pizza_olaf_llm/len(pizza_onto_obj_prop_uri)
objectProperties_recall_olafNoLLM = common_obj_prop_pizza_olaf_no_llm/len(pizza_onto_obj_prop_uri)

subClassOfPairs_precision_text2owl = common_sub_pizza_text2owl/len(llm_text2owl_sub_uri)
subClassOfPairs_precision_olafLLM = common_sub_pizza_olaf_llm/len(olaf_llm_sub_uri)
subClassOfPairs_precision_olafNoLLM = common_sub_pizza_olaf_no_llm/len(olaf_no_llm_sub_uri)

subClassOfPairs_recall_text2owl = common_sub_pizza_text2owl/len(pizza_onto_sub_uri)
subClassOfPairs_recall_olafLLM = common_sub_pizza_olaf_llm/len(pizza_onto_sub_uri)
subClassOfPairs_recall_olafNoLLM = common_sub_pizza_olaf_no_llm/len(pizza_onto_sub_uri)

In [47]:
def f1_score(precision, recall) -> float:
    if precision + recall == 0:
        return 0
    else:
        return 2*(precision * recall) / (precision + recall)

In [48]:
onto_evaluation = {
    "Metrics": [
        "Classes precision", 
        "Classes recall", 
        "Classes F1-score",
        "Individuals precision", 
        "Individuals recall", 
        "Individuals F1-score",
        "Classes and individuals precision", 
        "Classes and individuals recall", 
        "Classes and individuals F1-score",
        "Object properties precision", 
        "Object properties recall", 
        "Object properties F1-score",
        "SubClassOf pairs precision", 
        "SubClassOf pairs recall",
        "SubClassOf pairs F1-score"
    ],
    "Text to OWL": [
        classes_precision_text2owl, 
        classes_recall_text2owl, 
        f1_score(classes_precision_text2owl, classes_recall_text2owl),
        individuals_precision_text2owl,
        individuals_recall_text2owl, 
        f1_score(individuals_precision_text2owl, individuals_recall_text2owl),
        classesAndIndividuals_precision_text2owl, 
        classesAndIndividuals_recall_text2owl, 
        f1_score(classesAndIndividuals_precision_text2owl, classesAndIndividuals_recall_text2owl),
        objectProperties_precision_text2owl, 
        objectProperties_recall_text2owl, 
        f1_score(objectProperties_precision_text2owl, objectProperties_recall_text2owl),
        subClassOfPairs_precision_text2owl, 
        subClassOfPairs_recall_text2owl,
        f1_score(subClassOfPairs_precision_text2owl, subClassOfPairs_recall_text2owl,)
    ],
    "OLAF LLM": [
        classes_precision_olafLLM, 
        classes_recall_olafLLM, 
        f1_score(classes_precision_olafLLM, classes_recall_olafLLM),
        individuals_precision_olafLLM, 
        individuals_recall_olafLLM, 
        f1_score(individuals_precision_olafLLM, individuals_recall_olafLLM),
        classesAndIndividuals_precision_olafLLM, 
        classesAndIndividuals_recall_olafLLM, 
        f1_score(classesAndIndividuals_precision_olafLLM, classesAndIndividuals_recall_olafLLM),
        objectProperties_precision_olafLLM, 
        objectProperties_recall_olafLLM, 
        f1_score(objectProperties_precision_olafLLM, objectProperties_recall_olafLLM),
        subClassOfPairs_precision_olafLLM,
        subClassOfPairs_recall_olafLLM,
        f1_score(subClassOfPairs_precision_olafLLM, subClassOfPairs_recall_olafLLM)
    ],
    "OLAF no LLM": [
        classes_precision_olafNoLLM,
        classes_recall_olafNoLLM,
        f1_score(classes_precision_olafNoLLM, classes_recall_olafNoLLM),
        individuals_precision_olafNoLLM,
        individuals_recall_olafNoLLM,
        f1_score(individuals_precision_olafNoLLM, individuals_recall_olafNoLLM),
        classesAndIndividuals_precision_olafNoLLM,
        classesAndIndividuals_recall_olafNoLLM,
        f1_score(classesAndIndividuals_precision_olafNoLLM, classesAndIndividuals_recall_olafNoLLM),
        objectProperties_precision_olafNoLLM,
        objectProperties_recall_olafNoLLM,
        f1_score(objectProperties_precision_olafNoLLM, objectProperties_recall_olafNoLLM),
        subClassOfPairs_precision_olafNoLLM,
        subClassOfPairs_recall_olafNoLLM,
        f1_score(subClassOfPairs_precision_olafNoLLM, subClassOfPairs_recall_olafNoLLM)
    ]
}

df_evaluation = pd.DataFrame(onto_evaluation)

In [49]:
df_evaluation

,Metrics,Text to OWL,OLAF LLM,OLAF no LLM
0,Classes precision,1.000000,0.567010,0.387387
1,Classes recall,0.378947,0.578947,0.452632
2,Classes F1-score,0.549618,0.572917,0.417476
3,Individuals precision,0.000000,0.010526,0.005848
4,Individuals recall,0.000000,0.200000,0.400000
5,Individuals F1-score,0.000000,0.020000,0.011527
6,Classes and individuals precision,0.539683,0.556701,0.130243
7,Classes and individuals recall,0.239437,0.380282,0.415493
8,Classes and individuals F1-score,0.331707,0.451883,0.198319
9,Object properties precision,0.250000,0.064935,0.136364
